In [48]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255)
# imagedatagenerator 可以平移旋转放大缩小增减亮，用一张图片生成不同的图片来更好的训练模型
train_data = train_datagen.flow_from_directory('C:/非系统/dogs-vs-cats/train/train',target_size=(50,50),batch_size = 32,class_mode = 'binary')
# batch 是一批数据的大小
#注意地址必须是包含图片的类别文件夹的目录

Found 25000 images belonging to 2 classes.


In [49]:
# set up the cnn model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Conv2D,MaxPool2D,Flatten
model = Sequential()
#卷积层
model.add(Conv2D(32,(3,3),input_shape=(50,50,3),activation = 'relu'))
#池化层
model.add(MaxPool2D(pool_size = (2,2)))
#卷积层
model.add(Conv2D(32,(3,3),activation = 'relu'))
#池化层
model.add(MaxPool2D(pool_size = (2,2)))
#展开层
model.add(Flatten())
#全连接层
model.add(Dense(units = 128, activation = 'relu'))
model.add(Dense(units = 1, activation = 'sigmoid'))

In [50]:
model.compile(optimizer='adam',loss = 'binary_crossentropy',metrics='accuracy')
model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_10 (Conv2D)           (None, 48, 48, 32)        896       
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 24, 24, 32)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 22, 22, 32)        9248      
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 11, 11, 32)        0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 3872)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 128)               495744    
_________________________________________________________________
dense_5 (Dense)              (None, 1)                

In [51]:
#train the model
model.fit_generator(train_data,epochs = 10)
#因为是从ImagedataGenerator生成的，所以不是fit

Epoch 1/10
782/782 [==============================] - 61s 78ms/step - loss: 0.5790 - accuracy: 0.6846
Epoch 2/10
782/782 [==============================] - 60s 76ms/step - loss: 0.4746 - accuracy: 0.7724
Epoch 3/10
782/782 [==============================] - 59s 75ms/step - loss: 0.4321 - accuracy: 0.8010
Epoch 4/10
782/782 [==============================] - 58s 75ms/step - loss: 0.3931 - accuracy: 0.8193
Epoch 5/10
782/782 [==============================] - 57s 73ms/step - loss: 0.3496 - accuracy: 0.8443
Epoch 6/10
782/782 [==============================] - 59s 75ms/step - loss: 0.3046 - accuracy: 0.8691
Epoch 7/10
782/782 [==============================] - 58s 74ms/step - loss: 0.2556 - accuracy: 0.8917
Epoch 8/10
782/782 [==============================] - 54s 69ms/step - loss: 0.1991 - accuracy: 0.9180
Epoch 9/10
782/782 [==============================] - 54s 69ms/step - loss: 0.1377 - accuracy: 0.9468
Epoch 10/10
782/782 [==============================] - 56s 71ms/step - loss: 0.094

In [38]:
accuracy_train = model.evaluate(train_data)
print(train_data)
# 由于文件夹的字母表排列，cat排在dog前面，因此0是cat，1是dog，或者使用命令获知


782/782 [==============================] - 51s 65ms/step - loss: 3.2931e-12 - accuracy: 1.0000


In [57]:
print(train_data.class_indices)  #注意class_indices是flow_from_directory的属性

{'cat': 0, 'dog': 1}


In [89]:
#加载test1的所有图片
from tensorflow.keras.preprocessing.image import load_img, img_to_array
pic_dog = 'C:/非系统/dogs-vs-cats/test1/test1/885.jpg'
pic_dog = load_img(pic_dog,target_size = (50,50))
pic_dog = img_to_array(pic_dog)
pic_dog = pic_dog / 255
pic_dog = pic_dog.reshape(1,50,50,3)
result = model.predict_classes(pic_dog)
print(result)

[[1]]


1. 高光错误
2. 有很多人的干扰 - 错误
3. 遮挡 - 错误
4. 穿衣戴帽 - 错误
5. 简笔画 - 错误


To be completed
1. 将train 分割成train 和 test
2. 将test里的图片自动多张导入，生成图片显示和预测结果显示